## **Concurrencia y Paralelismo**

- Son conceptos relacionados, pero con diferencias clave.
- Ambos se utilizan para mejorar el rendimiento de las aplicaciones, especialmente cuando se trata de ejecutar tareas múltiples al mismo tiempo.

**Concurrencia**: Se refiere a la capacidad de ejecutar múltiples tareas en aparente simultaneidad. Puede implicar que las tareas se intercalen entre sí (sin necesariamente ejecutarse al mismo tiempo), pero aún así, la aplicación sigue respondiendo mientras se manejan múltiples tareas. Python utiliza hilos (threads) y procesos para manejar la concurrencia.

**Paralelismo**: Se refiere a la ejecución real de múltiples tareas al mismo tiempo, normalmente en máquinas con múltiples núcleos de CPU. Esto implica que las tareas se ejecutan en paralelo, distribuidas a través de diferentes procesadores.

## **Concurrencia**

La librería `threading` permite la creación y manejo de hilos (threads), que son unidades de ejecución dentro de un proceso. El uso de hilos permite que una aplicación realice múltiples tareas al mismo tiempo, aunque debido al `Global Interpreter Lock (GIL)` en CPython, los hilos no pueden ejecutar código Python puro en paralelo en múltiples núcleos. Sin embargo, los hilos son útiles para tareas I/O-bound, como operaciones de red o lectura/escritura de archivos, donde la mayor parte del tiempo el programa está esperando por datos.

- `threading.Thread` crea un hilo que ejecuta la función tarea().
- `start()` inicia el hilo.
- `join()` espera que el hilo termine antes de continuar con la ejecución.

In [ ]:
import threading
import time


def tarea():
    print("Tarea comenzada")
    time.sleep(3)
    print("Tarea completada")


# Crear hilos
hilo1 = threading.Thread(target=tarea)
hilo2 = threading.Thread(target=tarea)
hilo3 = threading.Thread(target=tarea)


# Iniciar los hilos
hilo1.start()
hilo2.start()
hilo3.start()


# Esperar que terminen
hilo1.join()
hilo2.join()
hilo3.join()


print("Ambas tareas han finalizado")

Una opción más avanzada y fácil de manejar para trabajar con hilos es el `ThreadPoolExecutor` en el módulo `concurrent.futures`. Este permite manejar un grupo de hilos (pool de hilos) y coordinar su ejecución.

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def tarea(x):
    print(f"Tarea {x} en ejecución")


# Crear un pool de 3 hilos
with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(tarea, range(5))  # Ejecuta tarea() para cada elemento en el rango


print("Todas las tareas han sido ejecutadas")

## **Paralelismo**

Para aprovechar el paralelismo real, donde se pueden ejecutar tareas en diferentes núcleos de CPU, puedes utilizar el módulo multiprocessing. Este crea procesos independientes, cada uno con su propio espacio de memoria, y puede aprovechar múltiples núcleos en un sistema, lo que permite realizar operaciones CPU-bound (que requieren mucho poder de procesamiento) en paralelo.

- `multiprocessing.Process` crea un proceso que ejecuta la función tarea().
- `start()` inicia el proceso.
- `join()` espera que el proceso termine antes de continuar.

In [ ]:
import multiprocessing

def tarea():
    print("Tarea ejecutada en un proceso")


# Crear procesos
proceso1 = multiprocessing.Process(target=tarea)
proceso2 = multiprocessing.Process(target=tarea)


# Iniciar procesos
proceso1.start()
proceso2.start()


# Esperar que terminen
proceso1.join()
proceso2.join()


print("Ambos procesos han finalizado")

In [ ]:
from concurrent.futures import ProcessPoolExecutor


def tarea(x):
    print(f"Tarea {x} ejecutada en un proceso")

if __name__ == "__main__":
    # Crear un pool de procesos
    with ProcessPoolExecutor(max_workers=4) as executor:
        executor.map(tarea, range(5))  # Ejecuta tarea() en paralelo para cada elemento

    print("Todas las tareas han sido ejecutadas")

## **Tareas combinadas: Concurrencia y Paralelismo**

En algunos casos, podrías querer combinar concurrencia y paralelismo en tu aplicación. Por ejemplo, si tienes tareas I/O-bound (que usan threading) junto con tareas CPU-bound (que usan multiprocessing), puedes coordinar ambos tipos de procesamiento para mejorar el rendimiento.

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import time


def tarea_io(x):
    time.sleep(1)
    print(f"Tarea I/O {x} terminada")


def tarea_cpu(x):
    return x * x  # Ejemplo simple de tarea CPU-bound


if __name__ == "__main__":
    # Usando ThreadPoolExecutor para tareas I/O-bound
    with ThreadPoolExecutor(max_workers=3) as executor_io:
        executor_io.map(tarea_io, range(5))

    # 
    # Usando ProcessPoolExecutor para tareas CPU-bound
    with ProcessPoolExecutor(max_workers=4) as executor_cpu:
        results = list(executor_cpu.map(tarea_cpu, range(5)))
        print("Resultados de tareas CPU-bound:", results)